<a href="https://colab.research.google.com/github/rodrigoviannini/Analysis_Data/blob/main/optical_character_recognition_20230620.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SETTINGS

In [1]:
# Install
!pip install --upgrade azure-cognitiveservices-vision-computervision;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.2 MB/s eta 0:00:00


In [2]:
# Imports
import os
import sys
import time
import pandas as pd
import re
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

MICROSOFT AZURE

Projets: XXX

Object: IMAGE

Activity: OCR

Addicional: RegEX

In [3]:
# Credentials
subscription_key = 'INSERT AZURE KEY'
endpoint = 'https://visionleitura.cognitiveservices.azure.com/'
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

In [6]:
# Cleanup and exception list
def clean_ocr_text(ocr):
    """ This function performs text cleaning using regular expressions (RegEX) and replace function.
     This function performs text cleaning using regular expressions (RegEX) and replace function. """

    # Exception list
    del_ = ['Cabo', 'Caber', 'Cab', 'Cont', 'Est',
            'ey', 'Com', 'Proc', 'Cool', 'Ext',
            'Co', 'abo', 'OPR', 'Rst', 'Fst', 'nt',
            'Cahn', 'oi','Caha', 'Caho',
            'Cant', 'ANATE', 'ANAT', 'Et', 'Came',
            'CABO ÓPTICO', 'V TAL', 'VTAL', '\n',
            'Caha:', 'Caha: ', 'queLa']

    text = str(ocr)
    for item in del_:
        # Character replacement
        text = text.replace(str(item) , '').strip()
        text = text.replace(' ','')
        text = text.replace("&","8")
        text = text.replace("|","I")
        text = text.replace("×","X")
        text = text.replace("g","9")
        text = re.sub(r"[][().:;_\"'ـه₦-]","", text)

    return text

In [9]:
# Azure
def extract_azure_ocr(filepath):
    """ This function returns OCR (Optical character recognition) which is the process that
    converts text in image format to text in human-readable character format. """

    # Open the file in binary format for reading
    content = open(filepath, 'rb')
    try:
        # Call the Azure Computer Vision client to read the file content
        # Set the raw parameter to True to return the raw response
        read_response = computervision_client.read_in_stream(content, raw=True)

        # Get the operation location from the response headers
        read_operation_location = read_response.headers["Operation-Location"]

        # Extract the operation ID from the operation location
        operation_id = read_operation_location.split("/")[-1]

        # Loop to check the status of the read operation
        while True:
            # Get the read result using the operation ID
            read_result = computervision_client.get_read_result(operation_id)

            # Check if the status is not 'notstarted' or 'running' to exit the loop
            if read_result.status.lower() not in ['notstarted', 'running']:
                break

            # Wait for a short interval before checking the status again
            time.sleep(0.2)

        if read_result.status == OperationStatusCodes.succeeded:
            # Initialize an empty string to store the extracted text
            texts = ''

            # Iterate over the read results and extract the text from each line found
            for text_result in read_result.analyze_result.read_results:
                for line in text_result.lines:
                    texts = texts + ' ' + line.text

            # Call the clean_ocr_text function to clean the extracted text
            return clean_ocr_text(texts)
        else:
            # If the operation status is not successful, print a message indicating the unexpected status
            print('Azure: unexpected status (', read_result.status, ') for file:', filepath)
    except:
        # If an exception occurs during the OCR request process, print a failure message
        print('Azure: fail to request OCR for file:', filepath)

    # Return an empty string if no OCR is extracted or an issue occurs
    return ''


In [8]:
image = 'add your image path'
displaying_text = extract_azure_ocr(image)
print(displaying_text)